# User-Defined Functions and Procedures

You can create custom user-defined functions and procedures for your Splice Machine database instance, and then call them as needed from your SQL. This notebook walks you through creating, using, and modifying functions and procedures that you can use for complex database queries. We cover the following topics:

* *Functions vs. Procedures*
* *Your Development Environment*
* *Local Development of the Custom Code in an IDE*
* *Jar File Deployment*
* *Install and Define Your Functions and Procedures*
* *Running Your User-Defined Functions and Procedures*


<p class="noteIcon">You may need to adapt some of the particulars described in this notebook to your own development environment.</p>


## Functions vs. Procedures

First, let’s define what a function is, what a procedure is, and their similarities and differences.

Functions are executed as part of an SQL statement. For example: 

```
SELECT MAX(salary) FROM employees
```

In this code, `max()` is function that is being executed.

Procedures are executed on their own with the `CALL` command.  For example: 

```
CALL SYSCS_UTIL.VACUUM()
```

Functions can be executed from a trigger, from an SQL SELECT statement, or with an SQL expression; however, functions are more restricted than procedures because their use is restricted, as summarized in the following table:

<table class="splicezep">
    <thead>
        <tr>
            <th>Feature</th>
            <th>Functions</th>
            <th>Procedures</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="ItalicFont">Execute in a trigger</td>
            <td>yes</td>
            <td>yes</td>
        </tr>
            <td class="ItalicFont">Return result set(s)</td>
            <td>no</td>
            <td>yes</td>
        </tr>
        <tr>
            <td class="ItalicFont">Process OUT / INOUT params</td>
            <td>no</td>
            <td>yes</td>
        </tr>
        <tr>
            <td class="ItalicFont">Execute SQL select</td>
            <td>yes</td>
            <td>yes</td>
        </tr>
        </tr>
            <td class="ItalicFont">Execute SQL update/insert/delete</td>
            <td>no</td>
            <td>yes</td>
        </tr>
        <tr>
            <td class="ItalicFont">Execute DDL (create/drop)</td>
            <td>no</td>
            <td>yes</td>
        </tr>
        <tr>
            <td class="ItalicFont">Execute in an SQL expression</td>
            <td>yes</td>
            <td>no</td>
        </tr>
    </tbody>
</table>

## Your Development Environment

The *environment* in which your custom function or stored procedure code is developed may change depending upon your need; we recommend that you flesh out the actual functionality of your custom code in a local (standalone) Splice Machine environment. Once you have refined your code and it behaves to your satisfaction in your local environment, you can re-deploy the developed code to your target instance. 

This notebook covers all of these steps, though as you will see, these steps occur OUTSIDE of an actual Notebook when running in a local environment. The examples in this notebook also assume that you are running a docker image of Splice Machine. We provide you with the `docker` commands that you will need.

### Prerequisites

Since you'll be developing the custom functions and procedures on your local machine, there are some pre-requisite software requirements. You need to have:

* JDK 1.8
* An Interactive Development Environment (IDE) such as *IntelliJ*, *Eclipse*, or *VS Code*
* Maven 3.3.9+
 

## Local Development of the Custom Code in an IDE

As mentioned above, Splice Machine strongly recommends that you develop your custom Java code using an Integrated Development Environment (IDE) on your local machine. That way you can develop, test, and tune locally, before deploying to a full cluster. The product of your code should be a JDK 1.8-compatible JAR file.

We'll show you one example of a custom function and a custom procedure.

The following steps are for developing your custom code using the IntelliJ IDEA on MacOS:

1. _Set up your project_
2. _Configure it to output a JAR file_
3. _Code your function or procedure_
4. _Build your JAR file_
5. _Deploy and test your code_

### 1. Set Up Your Project

Follow these steps to set up your project in IntelliJ IDEA:

1. Select `File -> New -> Project...`
2. Choose `Java with 1.8` as the Project SDK
3. Skip creating from a template unless you prefer to use one
4. Give your project a name and location; for example: `CompanyACustom1`
5. If desired, create a new package (`File -> New -> Package`), such as `com.splicemachine.companya`

### 2. Configuring It to Output a JAR File

Now follow these steps in IntelliJ IDEA to build a JAR file:

1. Select `File -> Project Structure...`
2. Select `Artifacts`, and click `+`
3. Choose `JAR -> From modules with dependencies `
3. Check `Build on make` and then click `OK`

### 3. Code Your Function or Procedure

Follow these steps to create your code:

1. Create a file by right-clicking on your src or package folder, and selecting `New -> Java Class`
2. Paste in code like the following:

```
package com.splicemachine.companya;

import java.sql.SQLException;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.PreparedStatement;


public class Custom1 {

    public static String replacestr(String source, String find, String replace) {
        return source.replace(find, replace);
    }

    public static void customIncrement () throws SQLException {
        try (Connection con = DriverManager.getConnection("jdbc:default:connection")) {

            String incrementSQL = "update SPLICE.INCREMENTTEST set i = i + 1";

            try (PreparedStatement ps = con.prepareStatement(incrementSQL)) {
                int resultSet = ps.executeUpdate();
            } catch (SQLException e) {
                con.rollback();
                e.printStackTrace();
                throw e;
            }
        }
    }
}
```

### 4. Build Your JAR File

Finally, follow these steps to build your JAR file:

1. `Build -> Artifacts... JAR`
2. Use your command line interface to navigate to where the JAR file is created, e.g. out/artifacts/CompanyACustom1_jar/Custom1.jar

### 5. Deploy and Test Your Code

You are now ready to deploy your JAR file to test!


## Development Guidelines
Before we develop the code and deploy the JAR file, it is worth going through some simple best practices:

1. It's useful for code organization to use a package for the code, e.g. `com.splicemachine.yourcompanyname`.
2. Notice you don't need a `main` code block in your class, because you'll be directly calling the class's specific method.
3. The example we've shown is a stored procedure, which means it does not return any results. If you create a function, it _must_ return a result.
4. You need to ensure that your method throws `SQLException` if SQL is being executed.  Be sure to use `try/catch` blocks similar to those shown in the example, to deal with any possible run-time exceptions.


## Useful Docker Commands

This training notebook is hosted from a docker image and runs the standalone version of Splice Machine. To test and run your custom functions and procedures you will need to copy your jar file to the docker image. Here are some useful docker commands:

### List running docker containers

```
docker ps -a
```

### Connect to a container

```
docker exec -it <container-id> bash
```

The `container-id` running the training notebooks is `spliceserver`

### Copy a file to a container

```
docker cp <file-name> <container-id>:<path-in-container>
```

The `path-in-container` is the full path inside the container.


## JAR File Deployment

To deploy your jar to your Splice Machine instance, you call the `SQLJ.INSTALL_JAR` procedure; for example:

```
CALL SQLJ.INSTALL_JAR('<your-full-path-to>/Custom1.jar', 'SPLICE.Custom1', 0);
```

Which path  you specify in the `<your-full-path-to>` parameter depends on your deployment scenario, as described in the following table:

<table class="zep">
  <col />
  <col />
  <thead>
      <tr>
          <th>Scenario</th>
          <th>JAR File Location</th>
          <th><code>&lt;your-full-path-to&gt;</code> points to</th>
      </tr>
  </thead>
  <tbody>
      <tr>
          <td>Locally running instance</td>
          <td>Anywhere in your local filesystem.</td>
          <td>Your local filesystem.</td>
      </tr>
      <tr>
          <td>Docker instance</td>
          <td>Copied into the Docker container running Splice Machine.</td>
          <td>The Docker filesystem.</td>
      </tr>
      <tr>
          <td>Cluster instance</td>
          <td>Copied to HDFS.</td>
          <td>The HDFS location.</td>
      </tr>
  </tbody>
</table>


## Install and Define Your Functions and Procedures

Once the jar file has been deployed, you need to execute the commands below using the Splice Machine sqlshell command line interpreter, `splice>`.

<p class="noteNote">You must execute these commands in your Splice Machine environment, __not in Zeppelin__.</p>

To align to your custom function/procedure and environment:

1. Install your JAR file.
2. Define the custom procedures/functions.

### Install Your Jar File

Let's install the JAR file and modify our classpath to include the new JAR. For example:

```
CALL SQLJ.INSTALL_JAR('<your-full-path-to>/Custom1.jar', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
```

<p class="noteNote">You only need to execute these calls once for each JAR file that you're deploying.</p>

### Define the Custom Procedures and Functions

Next we need to define our custom functions and procedures in the database. For our example, we're defining 
the `replacestr` function and `customIncrement` procedure:

```
CREATE FUNCTION SPLICE.replacestr(val1 varchar(100), val2 varchar(100), val3 varchar(100)) RETURNS varchar(100)
LANGUAGE JAVA
PARAMETER STYLE JAVA
NO SQL
EXTERNAL NAME 'com.splicemachine.companya.Custom1.replacestr';

CREATE PROCEDURE SPLICE.customincrement() 
PARAMETER STYLE JAVA 
MODIFIES SQL DATA 
LANGUAGE JAVA 
DYNAMIC RESULT SETS 0 
EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';
```

<p class="noteIcon">Please refer to our <a href="https://doc.splicemachine.com/developers_fcnsandprocs_intro.html">documentation</a> for a more comprehensive explanation on how to create custom functions and procedures.</p>

## Running Your User-Defined Functions and Procedures

To test our example custom procedure, you can create a table on which the stored procedure will operate. For example:

```
create table splice.test (i int, v varchar(10));
insert into splice.test values (1,'foobar');
```

Let's try the `replacestr` function:

```
select splice.replacestr(v,'bar','ey') from splice.test;
```

And now let's see the stored procedure into action:

```
call splice.customincrement();
select * from splice.test;
```


## Where to Go Next
The next notebook in this class, [*Using spark-submit with the Native Spark DataSource*](./i.%20Using%20spark-submit%20with%20Native%20Spark%20Datasource.ipynb), shows you how to take advantage of the Splice Machine Native Spark DataSource when running apps with spark-submit.
